## Generating data for ML

In [1]:
import pandas as pd
from collections import Counter
import random 
import ultimate
import numpy as np
# define card set
suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
rank_values = {rank: i for i, rank in enumerate(ranks, start=2)}

deck = [{'rank': rank, 'suit': suit} for suit in suits for rank in ranks]

combinations = ["High Card", "One Pair", "Two Pair", "Three of a Kind", "Four of a Kind", 
                "Full House", "Straight", "Flush", "Straight Flush", "Royal Flush"]
combinations_values = {combination: i for i, combination in enumerate(combinations, start=1)}
# set ordered winning combinations
winning_hands = ["High Card", "One Pair", "Two Pair", "Three of a Kind", "Straight", "Flush", 
                "Full House", "Four of a Kind", "Straight Flush", "Royal Flush"]

winning_hand_ranks = {hand: i for i, hand in enumerate(winning_hands)}
#enumerate the deck
enumerated_deck = dict(enumerate(deck, start=1))
num_deck = list(range(1, 53))

In [2]:
# first round options
print(52*51*50*49 / 4)
# second round options
print(52*51*50*49*48*47*46 / 12)
# third round options
print(52*51*50*49*48*47*46*45*44 / 24)
enumerated_deck

1624350.0
56189515200.0
55627620048000.0


{1: {'rank': '2', 'suit': 'Hearts'},
 2: {'rank': '3', 'suit': 'Hearts'},
 3: {'rank': '4', 'suit': 'Hearts'},
 4: {'rank': '5', 'suit': 'Hearts'},
 5: {'rank': '6', 'suit': 'Hearts'},
 6: {'rank': '7', 'suit': 'Hearts'},
 7: {'rank': '8', 'suit': 'Hearts'},
 8: {'rank': '9', 'suit': 'Hearts'},
 9: {'rank': '10', 'suit': 'Hearts'},
 10: {'rank': 'J', 'suit': 'Hearts'},
 11: {'rank': 'Q', 'suit': 'Hearts'},
 12: {'rank': 'K', 'suit': 'Hearts'},
 13: {'rank': 'A', 'suit': 'Hearts'},
 14: {'rank': '2', 'suit': 'Diamonds'},
 15: {'rank': '3', 'suit': 'Diamonds'},
 16: {'rank': '4', 'suit': 'Diamonds'},
 17: {'rank': '5', 'suit': 'Diamonds'},
 18: {'rank': '6', 'suit': 'Diamonds'},
 19: {'rank': '7', 'suit': 'Diamonds'},
 20: {'rank': '8', 'suit': 'Diamonds'},
 21: {'rank': '9', 'suit': 'Diamonds'},
 22: {'rank': '10', 'suit': 'Diamonds'},
 23: {'rank': 'J', 'suit': 'Diamonds'},
 24: {'rank': 'Q', 'suit': 'Diamonds'},
 25: {'rank': 'K', 'suit': 'Diamonds'},
 26: {'rank': 'A', 'suit': 'Diamo

## For one player

In [3]:
# ante is 1, blind is 1, player bets 1 or 0 (return bet or 
# fold depending whether you will win or lose)
def decide_game_victor_and_calculate_rewards_for_bets(row):
    
    player_hand = [enumerated_deck[card] for card in row[0:7]]
    dealer_hand = [enumerated_deck[card] for card in row[2:]]

    player_combination = ultimate.get_best_hand(player_hand)
    dealer_combination = ultimate.get_best_hand(dealer_hand)

    player_rank = winning_hand_ranks[player_combination]
    dealer_rank = winning_hand_ranks[dealer_combination]

    victor = 0 # 0 = dealer, 1 = player
    
    if player_rank > dealer_rank:
        victor = 1
    elif player_rank == dealer_rank:
        result = ultimate.decider(player_combination, player_hand, 
                                  dealer_combination, dealer_hand)
        if result == "player":
            victor = 1
        elif result == "dealer":	
            victor = 0
        else:
            victor = random.randint(0, 1)	# need to decide about this
    else:
        victor = 0

    # check if ante is valid
    dealer_has_something = ultimate.dealer_has_pair_or_better(dealer_hand[:2], dealer_hand[2:])
    blind_won = ultimate.has_blind(1, player_combination) - 1 #how much blind got us

    # calculate rewards for first and second rounds (in third victory is already bet, defeat is fold)
    first_round = 0
    second_round = 0
    third_round = 0

    # if its a draw, we leave both values at zero
    if victor == 1:
        first_round = 4 + blind_won + (1 if dealer_has_something else 0)
        second_round = 2 + blind_won + (1 if dealer_has_something else 0)
        third_round = 1 + blind_won + (1 if dealer_has_something else 0)
    elif victor == 0:
        first_round = -6
        second_round = -4
        third_round = -3
    
    return [third_round, first_round, second_round]

# function for conversion back to cards for checking
def convert_to_cards(row):
    cards = row[:9]
    output = [(enumerated_deck[card]["rank"], enumerated_deck[card]["suit"])  for card in cards]
    return output


In [4]:
# full dataset
hands = 50000
multiplier_D = 4
multiplier_1 = 3
multiplier_2 = 3

header = ["C1", "C2", "R1", "R2", "R3", "R4", "R5", "D1", "D2"]
rows = []
""" for A in range(hands):
    num_deck_copy = num_deck.copy()
    player = [num_deck_copy.pop(random.randint(0, len(num_deck_copy) - 1)), 
              num_deck_copy.pop(random.randint(0, len(num_deck_copy) - 1))]
    
    if A % 100 == 0:
        print(A)
    for K in range(multiplier_1):
        num_deck_copy_copy = num_deck_copy.copy()	
        flop = [num_deck_copy_copy.pop(random.randint(0, len(num_deck_copy_copy) - 1)), 
                num_deck_copy_copy.pop(random.randint(0, len(num_deck_copy_copy) - 1)), 
                num_deck_copy_copy.pop(random.randint(0, len(num_deck_copy_copy) - 1))]
        for K2 in range(multiplier_2):
            num_deck_copy_copy_copy = num_deck_copy_copy.copy()
            river = [num_deck_copy_copy_copy.pop(random.randint(0, len(num_deck_copy_copy_copy) - 1)),
                     num_deck_copy_copy_copy.pop(random.randint(0, len(num_deck_copy_copy_copy) - 1))]
            for D in range(multiplier_D):
                num_deck_copy_copy_copy_copy = num_deck_copy_copy_copy.copy()
                dealer = [num_deck_copy_copy_copy_copy.pop(random.randint(0, len(num_deck_copy_copy_copy_copy) - 1)), 
                        num_deck_copy_copy_copy_copy.pop(random.randint(0, len(num_deck_copy_copy_copy_copy) - 1))]
                rows.append(player + flop + river + dealer) """
for A in range(hands):
    if A % 1000 == 0:
        print(f"Hand {A}")
    player = random.sample(num_deck, 2)
    for _ in range(multiplier_1):
        remaining_deck = [card for card in num_deck if card not in player]
        flop = random.sample(remaining_deck, 3)
        for _ in range(multiplier_2):
            used = set(player + flop)
            river = random.sample([card for card in num_deck if card not in used], 2)
            for _ in range(multiplier_D):
                used2 = set(player + flop + river)
                dealer = random.sample([card for card in num_deck if card not in used2], 2)
                rows.append(player + flop + river + dealer)
full_data_set = pd.DataFrame(rows, columns=header)
print("generated variables")
full_data_set[["Q3", "Q1", "Q2"]] = full_data_set.apply(decide_game_victor_and_calculate_rewards_for_bets, axis=1, result_type="expand")
print("rewards calculated")
exp_values_Q1 = full_data_set.groupby(["C1", "C2"], sort=False).mean().reset_index().drop(columns=["R1", "R2", "R3", "R4", "R5", "D1", "D2", "Q3", "Q2"])
exp_values_Q2 = full_data_set.groupby(["C1", "C2", "R1", "R2", "R3"], sort=False).mean().reset_index().drop(columns=["R4", "R5", "D1", "D2", "Q3", "Q1"])
exp_values_Q3 = full_data_set.groupby(["C1", "C2", "R1", "R2", "R3", "R4", "R5"], sort=False).mean().reset_index().drop(columns=["D1", "D2", "Q2", "Q1"])
print("mean")
exp_values_Q1["Y1"] = exp_values_Q1["Q1"].apply(lambda x: 1 if x > 0 else 0)
exp_values_Q2["Y2"] = exp_values_Q2["Q2"].apply(lambda x: 1 if x > 0 else 0)
exp_values_Q3["Y3"] = exp_values_Q3["Q3"].apply(lambda x: 1 if x > 0 else 0)
print("y decided")
data_for_first_round = exp_values_Q1.drop(columns=["Q1"])
data_for_second_round = exp_values_Q2.drop(columns=["Q2"])
data_for_third_round = exp_values_Q3.drop(columns=["Q3"])
#data_for_second = full_data_set[::10].drop(columns=["R4", "R5"]) #get without last two cards
#data_for_first = data_for_second[::10].drop(columns=["R1", "R2", "R3"])  #get only first two cards
#data_for_first

#testing
pd.set_option('display.max_rows', 20)  # Show up to 100 rows
#test_copy = full_data_set.copy()
#test_copy[["C1", "C2", "R1", "R2", "R3", "R4", "R5", "D1", "D2"]] = test_copy.apply(convert_to_cards, 
#                                                                                    axis=1, result_type="expand")
#test_copy,
np.save("data_for_first_round.npy", data_for_first_round.to_numpy())
np.save("data_for_second_round.npy", data_for_second_round.to_numpy())
np.save("data_for_third_round.npy", data_for_third_round.to_numpy())
#print(data_for_first_round)
#print(data_for_second_round)
#print(data_for_third_round)

Hand 0
Hand 1000
Hand 2000
Hand 3000
Hand 4000
Hand 5000
Hand 6000
Hand 7000
Hand 8000
Hand 9000
Hand 10000
Hand 11000
Hand 12000
Hand 13000
Hand 14000
Hand 15000
Hand 16000
Hand 17000
Hand 18000
Hand 19000
Hand 20000
Hand 21000
Hand 22000
Hand 23000
Hand 24000
Hand 25000
Hand 26000
Hand 27000
Hand 28000
Hand 29000
Hand 30000
Hand 31000
Hand 32000
Hand 33000
Hand 34000
Hand 35000
Hand 36000
Hand 37000
Hand 38000
Hand 39000
Hand 40000
Hand 41000
Hand 42000
Hand 43000
Hand 44000
Hand 45000
Hand 46000
Hand 47000
Hand 48000
Hand 49000
generated variables
rewards calculated
mean
y decided


## For n players

In [5]:
# ante is 1, blind is 1, player bets 1 or 0 (return bet or 
# fold depending whether you will win or lose)
def decide_game_victor_and_calculate_rewards_for_bets_n(row, n):
    
    player_hand = [enumerated_deck[card] for card in row[["C1_P1", "C2_P1", "R1", "R2", "R3", "R4", "R5"]]]
    dealer_hand = [enumerated_deck[card] for card in row[["R1", "R2", "R3", "R4", "R5", "D1", "D2"]]]

    player_combination = ultimate.get_best_hand(player_hand)
    dealer_combination = ultimate.get_best_hand(dealer_hand)

    victor = 0 # 0 = dealer, 1 = player
    
    if winning_hands.index(player_combination) > winning_hands.index(dealer_combination):
        victor = 1
    elif winning_hands.index(player_combination) == winning_hands.index(dealer_combination):
        result = ultimate.decider(player_combination, player_hand, 
                                  dealer_combination, dealer_hand)
        if result == "player":
            victor = 1
        elif result == "dealer":	
            victor = 0
        else:
            victor = 2	# need to decide about this
    else:
        victor = 0

    # check if ante is valid
    dealer_has_something = ultimate.dealer_has_pair_or_better(dealer_hand[:2], dealer_hand[2:])
    blind_won = ultimate.has_blind(1, player_combination) - 1 #how much blind got us

    # calculate rewards for first and second rounds (in third victory is already bet, defeat is fold)
    first_round = 0
    second_round = 0

    # if its a draw, we leave both values at zero
    if victor == 1:
        first_round = 4 + blind_won + (1 if dealer_has_something else 0)
        second_round = 2 + blind_won + (1 if dealer_has_something else 0)
    elif victor == 0:
        first_round = -6
        second_round = -4
    
    return [victor, first_round, second_round]

# function for conversion back to cards for checking
def convert_to_cards_n(row, n):
    cards = row[:(7 + 2 * n)]
    output = [(enumerated_deck[card]["rank"], enumerated_deck[card]["suit"])  for card in cards]
    return output


In [6]:
# full dataset
hands = 2
multiplier_1 = 2
multiplier_2 = 2
n = 2

header = [] # Cards j Player i
for i in range(n):
    header += ["C1_P" + str(i + 1), "C2_P" + str(i + 1)]
header += ["R1", "R2", "R3", "R4", "R5", "D1", "D2"]
print(header)
full_data_set = pd.DataFrame(columns=header)
for A in range(hands):
    players = []
    num_deck_copy = num_deck.copy()
    for i in range(n):
        players += [num_deck_copy.pop(random.randint(0, len(num_deck_copy) - 1)), 
                num_deck_copy.pop(random.randint(0, len(num_deck_copy) - 1))]
    dealer = [num_deck_copy.pop(random.randint(0, len(num_deck_copy) - 1)), 
            num_deck_copy.pop(random.randint(0, len(num_deck_copy) - 1))]
    for K in range(multiplier_1):
        num_deck_copy_copy = num_deck_copy.copy()	
        flop = [num_deck_copy_copy.pop(random.randint(0, len(num_deck_copy_copy) - 1)), 
                num_deck_copy_copy.pop(random.randint(0, len(num_deck_copy_copy) - 1)), 
                num_deck_copy_copy.pop(random.randint(0, len(num_deck_copy_copy) - 1))]
        for K2 in range(multiplier_2):
            num_deck_copy_copy_copy = num_deck_copy_copy.copy()
            river = [num_deck_copy_copy_copy.pop(random.randint(0, len(num_deck_copy_copy_copy) - 1)),
                     num_deck_copy_copy_copy.pop(random.randint(0, len(num_deck_copy_copy_copy) - 1))]
            row = players + flop + river + dealer
            print(row)
            full_data_set.loc[len(full_data_set)] = row
print("generated variables")
print(decide_game_victor_and_calculate_rewards_for_bets_n(full_data_set.iloc[0], n))
full_data_set[["Y3", "Q1", "Q2"]] = full_data_set.apply(lambda row: decide_game_victor_and_calculate_rewards_for_bets_n(row, n), axis=1, result_type="expand")

exp_values_Q1 = full_data_set.groupby(header[:2*n]).mean().reset_index().drop(columns=["R1", "R2", "R3", "R4", "R5", "D1", "D2", "Y3", "Q2"])
exp_values_Q2 = full_data_set.groupby(header[:2*n + 3]).mean().reset_index().drop(columns=["R4", "R5", "D1", "D2", "Y3", "Q1"])

exp_values_Q1["Y1"] = exp_values_Q1["Q1"].apply(lambda x: 1 if x > 0 else 0)
exp_values_Q2["Y2"] = exp_values_Q2["Q2"].apply(lambda x: 1 if x > 0 else 0)

data_for_first_round = exp_values_Q1.drop(columns=["Q1"])
data_for_second_round = exp_values_Q2.drop(columns=["Q2"])
data_for_third_round = full_data_set.drop(columns=["D1", "D2", "Q1", "Q2"])
#data_for_second = full_data_set[::10].drop(columns=["R4", "R5"]) #get without last two cards
#data_for_first = data_for_second[::10].drop(columns=["R1", "R2", "R3"])  #get only first two cards
#data_for_first

#testing
pd.set_option('display.max_rows', 20)  # Show up to 100 rows
test_copy = full_data_set.copy()
print(test_copy)
test_copy[header] = test_copy.apply(lambda row: convert_to_cards_n(row, n), 
                                    axis=1, result_type="expand")
test_copy,
print(data_for_first_round)
print(data_for_second_round)
print(data_for_third_round)

['C1_P1', 'C2_P1', 'C1_P2', 'C2_P2', 'R1', 'R2', 'R3', 'R4', 'R5', 'D1', 'D2']
[6, 28, 52, 24, 36, 17, 29, 2, 14, 31, 11]
[6, 28, 52, 24, 36, 17, 29, 34, 10, 31, 11]
[6, 28, 52, 24, 46, 29, 45, 27, 12, 31, 11]
[6, 28, 52, 24, 46, 29, 45, 21, 15, 31, 11]
[48, 35, 50, 47, 6, 28, 26, 29, 11, 9, 42]
[48, 35, 50, 47, 6, 28, 26, 52, 17, 9, 42]
[48, 35, 50, 47, 33, 18, 4, 17, 7, 9, 42]
[48, 35, 50, 47, 33, 18, 4, 40, 24, 9, 42]
generated variables
[0, -6, -4]
   C1_P1  C2_P1  C1_P2  C2_P2  R1  R2  R3  R4  R5  D1  D2  Y3  Q1  Q2
0      6     28     52     24  36  17  29   2  14  31  11   0  -6  -4
1      6     28     52     24  36  17  29  34  10  31  11   0  -6  -4
2      6     28     52     24  46  29  45  27  12  31  11   1   4   2
3      6     28     52     24  46  29  45  21  15  31  11   1   4   2
4     48     35     50     47   6  28  26  29  11   9  42   1   5   3
5     48     35     50     47   6  28  26  52  17   9  42   1   5   3
6     48     35     50     47  33  18   4  17   7   9